This script is the main flood AA script that we run prior to the rainy season beginning in October. This script evaluates the performance of the Global Flood Awareness System (GloFAS) against observation based on past forecasts. We use this information to choose statistically backed trigger for anticipatory action. The goal is to understand what percentage of ensemble members from the forecast are needed to implement action on the ground.

As usual we start by importing all relevant packages saved in our aa-env. 

In [ ]:
from tqdm import tqdm
import xarray as xr
import pandas as pd
import os
import glob
import numpy as np
import time

Section 1: define variables, paths and read in data

In this first section we will simply define the relevant paths to where our forecast, observed and metadata are stored and load the data into Python. Remember to download this data from the README file if you haven't already done so. 

Ensure that your main directory and country of choice are correct. You should already have all the folders set-up. The script will automatically create the output folder in which we will store any results produced.

In [ ]:
# Start timer to track progress
start_time = time.time()

In [ ]:
country = 'zimbabwe'  # Define country of interest
directory = '/Users/jamietowner/Documents/flood_aa_training/'  # Define directory

In [ ]:
# Define paths to data
forecast_data_directory = os.path.join(directory, country, "data/forecasts")
observed_data_directory = os.path.join(directory, country, "data/reanalysis")
metadata_directory = os.path.join(directory, country, "data/metadata")
output_directory = os.path.join(directory, country, "outputs")

# Create directories if they don't exist
os.makedirs(output_directory, exist_ok=True)

In [ ]:
# Define filenames
observed_data_file = "glofas_observations.csv"
station_info_file = "metadata_zim.csv"

In [ ]:
# Load the observed data and gauging stations metadata
observed_data_path = os.path.join(observed_data_directory, observed_data_file)
station_info_path = os.path.join(metadata_directory, station_info_file)
observed_data = pd.read_csv(observed_data_path)
station_info = pd.read_csv(station_info_path) 

The next two cells we can quickly check the data input is what we expected. We can use different ways to do this such as df.columns or df to visualise the column headers or see the entire dataframe.

In [ ]:
# Show column headers of dataframe 
observed_data.columns

In [ ]:
# Show dataframe 
station_info

In [ ]:
# Convert the date column in observed_data to pandas timestamps 
observed_data["date"] = pd.to_datetime(observed_data["date"])

In [ ]:
# Load all GloFAS forecast files
forecast_files = glob.glob(os.path.join(forecast_data_directory, '*.nc'))

In the below cell, we use what is know as a print command to tell Python we would like it to print the output of a variable. In this case the relevant data directories. Printing variables and paths is a good way to check everything is correct and a good way to debug simple errors in the code.

In [ ]:
# Print paths to ensure they are set correctly
print(f"""
forecast directory: {forecast_data_directory}
observed data directory: {observed_data_directory}
metadata directory: {metadata_directory}
output directory: {output_directory}
""")

Section 2: process observations and forecasts and define events/non-events

In this section we will be processing the observation and forecasts by looping over the different dimensions of the forecast data, such as the lead times (out to 46 days), ensemble members (11 for the hindcast data) and each of the forecast issue dates between 2003 and 2023. The script takes each forecast and defines the forecast issue from this information. It then finds the forecast end date (for each lead time) and extract the observed and forecasts value for this end date. It then evaluates whether or not this value (for both observed and forecasted data) exceeds the defined thresholds. If it does it assigns a 1 (or TRUE) and if it does not it assigns a 0 (or FALSE). 

The script finishes by creating a dictionary of all the forecast and stores information such as the forecast issue date, forecast end date, the threshold category, the ensemble number and if there was an observed or forecasted flood event or not (i.e., based on the 0's and 1's). Finally, we create a dataframe so we can manipulate this data to calculate our triggers further down in the script. 

For this training we will be processing three gauging stations and two thresholds (moderate and severe) as it can take quite a long time to run for all stations (up to a day!). We will be running this script for all stations on a server nearer to the time of the 2024/25 rainy season.

In [ ]:
# Create an empty list to store events/non-events 
events = []

In [ ]:
# Loop over each forecast file 
for forecast_file in tqdm(forecast_files, desc="Processing forecast files"):
    # Load the NetCDF file
    ds = xr.open_dataset(forecast_file)
    
    # Extract all ensemble members from the 'number' dimension
    ensemble_members = ds['dis24'].coords['number'].values  # Assumes 'number' is the dimension for ensemble members


    # Extract the forecast issue date from the file and convert to pandas datetime
    forecast_issue_ns = ds['time'].values.item()  # Get the single scalar value (nanoseconds since epoch)
    forecast_issue_date = pd.to_datetime(forecast_issue_ns, unit='ns')

    # Define the lead times
    lead_times = list(range(0, 15))  # 0 up to 46 days

    # Loop over station metadata to extract information for each station
    for index, station_row in station_info.iterrows():
        station_name = station_row["station name"]
        station_lat = station_row["lat"]
        station_lon = station_row["lon"]

        # Extract individual station thresholds from metadata file
        thresholds = {
           # "bankfull": (station_row["obs_bankfull"], station_row["glofas_bankfull"]),
            "moderate": (station_row["obs_moderate"], station_row["glofas_moderate"]),
            "severe": (station_row["obs_severe"], station_row["glofas_severe"]),
        }
        
        # Process each lead time
        for lead_time in lead_times:
            # Calculate the forecast end date based on lead time
            forecast_end_date = forecast_issue_date + pd.DateOffset(days=lead_time)
            
            # Filter observed data for the matching period
            observed_period = observed_data[observed_data["date"] == forecast_end_date]
            
            # Skip if no observation data is available for this period
            if observed_period.empty:
                continue

            observed_values = observed_period[station_name].values[0]

            # Skip if there's no observation data (NaN value) for the specific station
            if pd.isnull(observed_values):
                continue

            # Extract forecast values for each ensemble member
            for ensemble_member in ensemble_members:
                forecast_data = ds['dis24'].sel(number=ensemble_member).isel(step=lead_time).values

                # Loop over the thresholds
                for severity, (obs_threshold, sim_threshold) in thresholds.items():
                    # Define events and non-events
                    observed_event = observed_values > obs_threshold
                    forecast_event = forecast_data > sim_threshold
                    
                    # Define tolerant events and non-events (within ±2 days window)
                    observed_window = observed_data[(observed_data["date"] >= forecast_end_date - pd.DateOffset(days=2)) & 
                                                    (observed_data["date"] <= forecast_end_date + pd.DateOffset(days=2))]
                    tolerant_observed_event = (observed_window[station_name] > obs_threshold).any()
                    
                    # Create a dictionary to store results
                    events_dict = {
                        "forecast file": os.path.basename(forecast_file),
                        "lead time": lead_time,
                        "station name": station_name,
                        "ensemble member": ensemble_member,
                        "forecasted date": forecast_end_date.date(),
                        "threshold": severity,
                        "observed event": observed_event,
                        "forecast event": forecast_event,
                        "tolerant observed event": tolerant_observed_event
                    }
            
                    # Append the events dictionary to the list
                    events.append(events_dict)

# Create a data frame from the list of event dictionaries
events_df = pd.DataFrame(events)
print("Processing complete.")

Section 3: construct contigency table and skill score metrics

In this section we will be using the dataframe produced above to construct contigency table and verification metrics to assess the skill of our forecasts at each station for different lead times. 

The contigency table is based on the hits, misses, false alarms and correct rejections which allows us to calculate metrics such as hit and false alarm rates. This tells us based on past forecasts how well the system would have performed in the past. If the forecast is very good then the hit rate will be closer to 1 and if it is particularly bad it will be closer to 0. We aim to have a hit rate above 0.5 for each station. 

As we are using ensemble data we have 11 predicted values at each lead time. When analysing the table we have TRUE or FALSE (or 1's and 0's) for the observed event and then 11 values of TRUE or FALSE. To make it easier to analyse we first calculate what is called the probability of detection. The is simply summing the number of 1's (i.e., hits) in a single forecasts from the 11 members and then diving by the total number of ensemble members (11 in our case). This provides us with a percentage which show how likely or unlikely the forecast thought a potential flood could be. This is key when analysing and choosing triggers further down the script.  

In [ ]:
# pivot the events_df to list ensemble members as columns
pivot_events_df = events_df.pivot_table(index=["forecast file", "lead time", "station name", "forecasted date","threshold", "observed event", "tolerant observed event"],
                                        columns="ensemble member",)

In [ ]:
# reset index to convert the pivoted dataframe to a flat table structure
pivot_events_df.reset_index(inplace=True)

In [ ]:
# define the columns corresponding to the forecast ensemble members (5 to 16)
ensemble_member_columns = pivot_events_df.columns[7:18]

In [ ]:
# calculate the percentage of ensemble members with events (i.e., 1's) for each row
pivot_events_df["probability of detection"] = pivot_events_df[ensemble_member_columns].sum(axis=1) / len(ensemble_members)

In [ ]:
# Generate trigger thresholds from 0.01 to 0.99 with a step size of 0.01
thresholds = np.arange(0.01, 1, 0.01)

In [ ]:
# initialize an empty dataframe to store the event occurrence for each threshold
metrics_df = pd.DataFrame(index=pivot_events_df.index)

The next part of the script loops over the thresholds variable which we just defined from 0.01 to 1. It analyses each forecast to see if the probability of detection exceeds each triggers value. This is the same step that we performed yesterday in the excel session. 

In [ ]:
# loop over each threshold
for threshold in thresholds:
    # determine if the forecast event occurs based on the threshold
    event_occurrence = (pivot_events_df['probability of detection'] >= threshold).astype(int)
    
    # add the event occurrence as a column in the event_df DataFrame
    metrics_df[f'threshold_{threshold:.2f}'] = event_occurrence

# concatenate the forecast file and observed event columns from result_df to event_df
metrics_df = pd.concat([pivot_events_df[['forecast file','station name', 'lead time','forecasted date','threshold','observed event','tolerant observed event','probability of detection']], metrics_df], axis=1)

# flatten the multi-index columns
metrics_df.columns = [''.join(col).strip() for col in metrics_df.columns.values]

In the next cell we define different ranges of lead times (e.g., 10-15 days) in order to evaluate the performance of the forecast at different timescales and see how far we can skillfully predict into the future. We then group our huge dataframe of all stations, lead times, thresholds and filter it into lots of smaller dataframes based on the station name, lead time category and threshold. We can then evaulate each station easier and in more detail. 

In [ ]:
# define the lead time ranges (optional, change as desired)
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 46, float('inf')]
labels = ['0-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-46', '0-46']

# create a new column called lead time category' that categorizes lead times into these ranges
metrics_df['lead time category'] = pd.cut(metrics_df['lead time'], bins=bins, labels=labels, right=False)

# group by station name, lead time category, and threshold
grouped = metrics_df.groupby(['station name', 'lead time category', 'threshold'], observed=False)

# create a dictionary to store each group's dataframe
grouped_dfs = {name: group for name, group in grouped}

The next cell create a function called 'calculate_metrics' and this is used to evaluate each of the individually filtered grouped dataframes which we created above. In this function we will calculate various metrics such as the number of hits, false alarms, misses, correct rejection, hit and false alarm rates and some metrics such as the critical success index (CSI). The CSI tells us how well the forecasting system balances the hit rate and false alarm ratio and is a key indicator for anticipatory action systems for determining triggers (i.e., what percentage of ensemble members will we be willing to implement anticipatory action on).

In [ ]:
# function to calculate verification metrics metrics for each grouped dataframe 
def calculate_metrics(df):
    hits = {}
    false_alarms = {}
    misses = {}
    correct_rejections = {}
    hit_rate = {}
    false_alarm_rate = {}
    csi = {}
    pss = {}
    f1_scores = {}
    

    # calculate contigency table metrics for each trigger threshold column
    for column in df.columns[8:107]:  # check columns from index 8 onwards are thresholds
        hits[column] = ((df['observed event'] == 1) & (df[column] == 1)).sum()
        false_alarms[column] = ((df['observed event'] == 0) & (df[column] == 1)).sum()
        misses[column] = ((df['observed event'] == 1) & (df[column] == 0)).sum()
        correct_rejections[column] = ((df['observed event'] == 0) & (df[column] == 0)).sum()
       
        # calculate verification metrics
        total_observed_events = hits[column] + misses[column]
        total_forecasted_events = hits[column] + false_alarms[column]
        hit_rate[column] = hits[column] / total_observed_events if total_observed_events > 0 else 0
        false_alarm_rate[column] = false_alarms[column] / total_forecasted_events if total_forecasted_events > 0 else 0
        csi[column] = hits[column] / (hits[column] + false_alarms[column] + misses[column]) if (hits[column] + false_alarms[column] + misses[column]) > 0 else 0
        pss[column] = (hits[column] * correct_rejections[column] - false_alarms[column] * misses[column]) / \
                      ((hits[column] + misses[column]) * (false_alarms[column] + correct_rejections[column])) if (hits[column] + misses[column]) * (false_alarms[column] + correct_rejections[column]) > 0 else 0
        precision = hits[column] / total_forecasted_events if total_forecasted_events > 0 else 0
        recall = hit_rate[column]
        f1_scores[column] = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # convert the metrics dictionaries to dataframes
    hits_df = pd.DataFrame(hits, index=['hits'])
    false_alarms_df = pd.DataFrame(false_alarms, index=['false alarms'])
    misses_df = pd.DataFrame(misses, index=['misses'])
    correct_rejections_df = pd.DataFrame(correct_rejections, index=['correct rejections'])
    hit_rate_df = pd.DataFrame(hit_rate, index=['hit rate'])
    false_alarm_rate_df = pd.DataFrame(false_alarm_rate, index=['false alarm rate'])
    
    csi_df = pd.DataFrame(csi, index=['csi'])
    pss_df = pd.DataFrame(pss, index=['pss'])
    f1_scores_df = pd.DataFrame(f1_scores, index=['f1 score'])
 
    # concatenate the metrics dataframes
    metrics_df = pd.concat([
        hits_df, false_alarms_df,
        misses_df, correct_rejections_df, hit_rate_df, false_alarm_rate_df,
        csi_df, pss_df, f1_scores_df,
    ])

    return metrics_df

The cell below simply runs the function above over the group of dataframes that we created.

In [ ]:
# iterate through each dataframe in grouped_dfs, apply calculate_metrics, and store the results back into grouped_dfs
for key, df in grouped_dfs.items():
    grouped_dfs[key] = calculate_metrics(df)
    
    # timer end 
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"elapsed time for calculating metrics: {elapsed_time:.2f} seconds")

To view the results of a specifc dataframe (i.e., the performance of one of the stations) run the cell below. Feel free to change the key inputs to one of the other stations, lead time categories or thresholds.

In [ ]:
# view a specific grouped dataframe 
specific_df = grouped_dfs["Nyakapupu",'0-5','moderate']
specific_df

Section 4: choosing triggers

In this section we will be choosing triggers based on the results from each specific dataframes above. The code loops through each dataframe and finds the percentage trigger where the CSI metric is the highest. In the case of multiple triggers sharing the highest CSI it then looks to choose the metric with the highest hit rate. You can visualise the output of the dataframe in your output directory as a csv. 

In [ ]:
# Create an empty dictionary to store the best triggers
best_triggers = {}

In [ ]:
# Iterate through each dataframe in grouped_dfs
for key, df in grouped_dfs.items():
    # Find the column names corresponding to thresholds
    threshold_columns = [col for col in df.columns if col.startswith('threshold_')]
    
    # Determine the maximum CSI value for each trigger
    max_csi = df.loc['csi'].max()
    
    # Find all trigger with the maximum CSI value
    best_csi_trigger = df.loc['csi'][df.loc['csi'] == max_csi].index.tolist()
    
    # If there are multiple triggers with the same CSI, choose the one with the highest hit rate
    if len(best_csi_trigger) > 1:
        # Find the hit rate for each of the best CSI triggers
        hit_rates = df.loc['hit rate'][best_csi_trigger]
        max_hit_rate = hit_rates.max()
        
        # Find all triggers with the maximum hit rate
        best_hit_rate_trigger = hit_rates[hit_rates == max_hit_rate].index.tolist()
        
        # If there are still ties, choose the trigger with the lowest value
        if len(best_hit_rate_trigger) > 1:
            # Convert trigger names to numbers (assuming trigger names follow a specific format)
            numeric_trigger = [float(thresh.split('_')[1]) for thresh in best_hit_rate_trigger]
            best_trigger = best_hit_rate_trigger[np.argmin(numeric_trigger)]  # Get the lowest threshold
        else:
            best_trigger = best_hit_rate_trigger[0]
    else:
        best_trigger = best_csi_trigger[0]
    
    # Store the best trigger information
    best_triggers[key] = {
        'best_trigger': best_trigger,
        'csi_value': df.loc['csi', best_trigger],
        'hit_rate': df.loc['hit rate', best_trigger]
    }

# Convert the best_triggers dictionary to a DataFrame for better readability
best_triggers_df = pd.DataFrame(best_triggers).T

# Print the DataFrame with the best triggers
print(best_triggers_df)


In [ ]:
# Convert the best_triggers dictionary to a DataFrame for better readability
best_triggers_df = pd.DataFrame(best_triggers).T
best_triggers_df

In [ ]:
# Save the results to a CSV file
best_triggers_df.to_csv(os.path.join(output_directory, f'triggers_{country}.csv'))

Congratulations! you have now completed the preseason script for the flood AA system. As mentioned we will be running this for all stations and thresholds prior to the rainy season in October 2025. We are now currently working on the operational script which will be used to monitor the daily forecasts from GloFAS to see if the triggers have been exceeded. This information will be updated in an online dashboard using PRISM similar to the drought AA set-up. 